# LLM-Blender Usage examples

## Loading blender (quick start)
You can find more custom configurations in 
- PairRanker: [./llm_blender/pair_ranker/config.py](./llm_blender/pair_ranker/config.py)
- GenFuser: [./llm_blender/gen_fuser/config.py](./llm_blender/gen_fuser/config.py)
- Blender: [./llm_blender/blender/config.py](./llm_blender/blender/config.py)

In [1]:
import os
import llm_blender_mps as llm_blender

blender = llm_blender.Blender()
# Load Ranker
# blender.loadranker("llm-blender/PairRM") # load ranker checkpoint
blender.loadranker("OpenAssistant/reward-model-deberta-v3-large-v2") # load ranker checkpoint
# Load Fuser
# blender.loadfuser("llm-blender/gen_fuser_3b") # load fuser checkpoint if you want to use pre-trained fuser; or you can use ranker only

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/3.88k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/455 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

/Users/fangyuanyu/anaconda3/lib/python3.11/site-packages/dataclasses_json/core.py:188: RuntimeWarning: 'NoneType' object value of non-optional type cache_dir detected when decoding RankerConfig.
  warnings.warn(
/Users/fangyuanyu/anaconda3/lib/python3.11/site-packages/dataclasses_json/core.py:188: RuntimeWarning: 'NoneType' object value of non-optional type load_checkpoint detected when decoding RankerConfig.
  warnings.warn(
/Users/fangyuanyu/anaconda3/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:515: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/Users/fangyuanyu/anaconda3/lib/python3.11/site-package

Successfully loaded ranker from  /Users/fangyuanyu/.cache/huggingface/hub/OpenAssistant/reward-model-deberta-v3-large-v2


In [28]:
# Hacking into MPS compatible mode
from llm_blender.blender.config import BlenderConfig
device = torch.device("mps")
blender_config = BlenderConfig(device=torch.device("mps"))
blender = llm_blender.Blender(blender_config=blender_config)
blender.loadranker("llm-blender/PairRM", device=device) # load ranker checkpoint

Successfully loaded ranker from  /Users/fangyuanyu/.cache/huggingface/hub/llm-blender/PairRM


In [29]:
inputs = ["hello, how are you!", "I love you!"]
candidates_texts = [["get out!", "hi! I am fine, thanks!", "bye!"], 
                    ["I love you too!", "I hate you!", "Thanks! You're a good guy!"]]
ranks = blender.rank(inputs, candidates_texts, return_scores=False, batch_size=1)
print(ranks)

Ranking candidates: 100%|██████████| 2/2 [00:00<00:00,  2.37it/s]

[[3 1 2]
 [1 3 2]]


## Load Mixinstruct dataset for the following examples showing

In [2]:
import datasets
import json
from llm_blender_mps.gpt_eval.cor_eval import COR_MAPS
from llm_blender_mps.gpt_eval.utils import get_ranks_from_chatgpt_cmps
mixinstruct_test = datasets.load_dataset("llm-blender/mix-instruct", split="test", streaming=True)
few_examples = list(mixinstruct_test.take(8))
# remove cmp_results with none cmp results
for ex in few_examples:
    ex['cmp_results'] = json.loads(ex['cmp_results'])
few_examples = [x for x in few_examples if x['cmp_results']]
insts = [x['instruction'] for x in few_examples]
inputs = [x['input'] for x in few_examples]
candidates_texts = [[cand['text'] for cand in x['candidates']] for x in few_examples]
print("Example:")
print("Instruction 1:\n", insts[0])
print("Input 1:\n", inputs[0])
print("Candidate 1 for input 1:\n")
print(candidates_texts[0][0])

ValueError: Invalid pattern: '**' can only be an entire path component

## Use case 1: Using LLM-Blender for ranking
By the rank function, LLM-Blender could ranks the candidates through pairwise comparisons and return the ranks. We show our ranker's ranks are highly correlated with ChatGPT ranks.

In [ ]:
ranks = blender.rank(inputs, candidates_texts, instructions=insts, return_scores=False, batch_size=2)

In [ ]:
print("Ranks for input 1:", ranks[0]) # ranks of candidates for input 1
# Ranks for input 1: [ 1 11  4  9 12  5  2  8  6  3 10  7]

In [ ]:
import numpy as np
llm_ranks_map, gpt_cmp_results = get_ranks_from_chatgpt_cmps(few_examples)
gpt_ranks = np.array(list(llm_ranks_map.values())).T
print("Correlation with ChatGPT")
print("------------------------")
for cor_name, cor_func in COR_MAPS.items():
    print(cor_name, cor_func(ranks, gpt_ranks))

## Use case 2: Using LLM-blender to directly compare two candidates

In [ ]:
candidates_A = [x['candidates'][0]['text'] for x in few_examples]
candidates_B = [x['candidates'][1]['text'] for x in few_examples]
comparison_results = blender.compare(
    inputs, candidates_A, candidates_B, instructions=insts, 
    batch_size=2, return_logits=False)
print("Comparison results for inputs:", comparison_results) # comparison results for input 1

## Use case 3: Using LLM-Blender for fuse generation
We show that the the fused generation using the top-ranked candidate from the rankers could get outputs of higher quality.

In [ ]:
from llm_blender.blender.blender_utils import get_topk_candidates_from_ranks
topk_candidates = get_topk_candidates_from_ranks(ranks, candidates_texts, top_k=3)
fuse_generations = blender.fuse(inputs, topk_candidates, instructions=insts, batch_size=2)
print("fuse_generations for input 1:", fuse_generations[0])

In [ ]:
# # Or do rank and fuser together
fuse_generations, ranks = blender.rank_and_fuse(inputs, candidates_texts, instructions=insts, return_scores=False, batch_size=2, top_k=3)

In [ ]:
from llm_blender.common.evaluation import overall_eval
metrics = ['bartscore']
targets = [x['output'] for x in few_examples]
scores = overall_eval(fuse_generations, targets, metrics)

print("Fusion Scores")
for key, value in scores.items():
    print("  ", key+":", np.mean(value))

print("LLM Scores")
llms = [x['model'] for x in few_examples[0]['candidates']]
llm_scores_map = {llm: {metric: [] for metric in metrics} for llm in llms}
for ex in few_examples:
    for cand in ex['candidates']:
        for metric in metrics:
            llm_scores_map[cand['model']][metric].append(cand['scores'][metric])
for i, (llm, scores_map) in enumerate(llm_scores_map.items()):
    print(f"{i} {llm}")
    for metric, llm_scores in llm_scores_map[llm].items():
        print("  ", metric+":", "{:.4f}".format(np.mean(llm_scores)))


## Use case 4: Use LLM-Blender for decoding enhancement (best-of-n sampling)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceH4/zephyr-7b-beta", device_map="auto")

system_message = {
    "role": "system",
    "content": "You are a friendly chatbot who always responds in the style of a pirate",
}
messages = [
    [   
        system_message,
        {"role": "user", "content": _inst + "\n" + _input},
    ]
    for _inst, _input in zip(insts, inputs)
]
prompts = [tokenizer.apply_chat_template(m, tokenize=False, add_generation_prompt=True) for m in messages]
outputs = blender.best_of_n_generate(model, tokenizer, prompts, n=10)
print("### Prompt:")
print(prompts[0])
print("### best-of-n generations:")
print(outputs[0])


## Use case 5: Use PairRM for RLHF tuning